In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
# import random

### Machine Learning

# transformation
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, OneHotEncoder

# models
from sklearn.model_selection import train_test_split

# metrics and evaluation
from scipy.stats import probplot

### Data Viz

# graphical basics
import matplotlib.pyplot as plt
%matplotlib inline

# graphical seaborn
import seaborn as sns
import folium as folium
# # graphical plotly
# import plotly.graph_objects as go
# import plotly.express as px
# # for jupyter notebook display management
# import plotly.io as pio
# pio.renderers.default = "notebook"

In [ ]:
df = pd.read_csv("C:/Users/elias/Downloads/comptage-velo-donnees-compteurs_preprocessed.csv", sep =',')

In [ ]:
# Extraire les composantes temporelles textuelles
df['date_et_heure_de_comptage'] = pd.to_datetime(df['date_et_heure_de_comptage'], utc=True).dt.tz_convert('Europe/Paris')
df['Jour'] = df['date_et_heure_de_comptage'].dt.day_name()
df['Mois'] = df['date_et_heure_de_comptage'].dt.month_name()
df['Heure'] = df['date_et_heure_de_comptage'].dt.hour

In [ ]:
# Corriger et uniformiser la colonne 'date_et_heure_de_comptage' si elle contient des objets ou chaînes
df['date_et_heure_de_comptage'] = pd.to_datetime(df['date_et_heure_de_comptage'], errors='coerce', utc=True).dt.tz_convert('Europe/Paris')

# Supprimer le fuseau horaire de la colonne pour afficher l'heure locale sans le "+01:00"
df['date_et_heure_de_comptage'] = df['date_et_heure_de_comptage'].dt.tz_localize(None)

# Vérification
df[['date_et_heure_de_comptage']].head()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# Nettoyage des noms de colonnes (suppression des espaces, remplacement par des underscores)
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("'", "")

# Conversion de la date et heure de comptage en datetime sans fuseau horaire visible
df['date_et_heure_de_comptage'] = pd.to_datetime(df['date_et_heure_de_comptage'])
df['date_et_heure_de_comptage'] = df['date_et_heure_de_comptage'].dt.tz_localize(None)


In [ ]:
# Création des colonnes temporelles

df['année'] = df['date_et_heure_de_comptage'].dt.year
df['mois'] = df['date_et_heure_de_comptage'].dt.month_name()
df['jour'] = df['date_et_heure_de_comptage'].dt.day_name()
df['heure'] = df['date_et_heure_de_comptage'].dt.hour
df['semaine'] = df['date_et_heure_de_comptage'].dt.isocalendar().week

In [ ]:
# Conversion des coordonnées en float
if 'coordonnées' in df.columns:
    df[['latitude', 'longitude']] = df['coordonnées'].str.split(',', expand=True).astype(float)

# Vérification des valeurs manquantes
print("--- Valeurs manquantes ---")
print(df.isnull().sum())

In [ ]:
# 1. Distribution du comptage horaire
plt.figure(figsize=(10,6))
sns.histplot(df['comptage_horaire'], bins=30, kde=True)
plt.title("Distribution du comptage horaire")
plt.xlabel("Comptage horaire")
plt.ylabel("Fréquence")
plt.show()

In [ ]:
# 4. Heatmap heure vs jour
plt.figure(figsize=(12,8))
pivot = df.pivot_table(index='heure', columns='jour', aggfunc='size', fill_value=0)
sns.heatmap(pivot, cmap='YlGnBu', annot=True, fmt=".0f")
plt.title("Nombre d'enregistrements par heure et jour")
plt.xlabel("Jour")
plt.ylabel("Heure")
plt.show()

In [ ]:
# 5. Boxplot du comptage horaire par jour
plt.figure(figsize=(12,6))
sns.boxplot(x='jour', y='comptage_horaire', data=df)
plt.title("Distribution du comptage horaire par jour")
plt.xlabel("Jour")
plt.ylabel("Comptage horaire")
plt.show()
plt.figure(figsize=(14,6))
sns.boxplot(x='heure', y='comptage_horaire', data=df)
plt.title("Comptage horaire par heure")
plt.show()

In [ ]:
# 6. Boxplot du comptage horaire par heure
plt.figure(figsize=(14,6))
sns.boxplot(x='heure', y='comptage_horaire', data=df)
plt.title("Distribution du comptage horaire par heure")
plt.xlabel("Heure")
plt.ylabel("Comptage horaire")
plt.show()

In [ ]:
# 6. Enregistrements par mois
plt.figure(figsize=(12,6))
mois_ordre = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
df['mois'] = pd.Categorical(df['mois'], categories=mois_ordre, ordered=True)
df['mois'].value_counts().sort_index().plot(kind='bar')
plt.title("Enregistrements par mois")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 11. Nuage de points comptage horaire vs heure
plt.figure(figsize=(12,6))
sns.stripplot(x='heure', y='comptage_horaire', data=df, jitter=0.2, alpha=0.4)
plt.title("Comptage horaire selon l'heure")
plt.show()

In [ ]:
# 15. Pairplot des variables temporelles et du comptage
sns.pairplot(df[['heure', 'semaine', 'comptage_horaire']])
plt.suptitle("Relation entre heure, semaine et comptage", y=1.02)
plt.show() 

In [ ]:
# 16. Répartition des valeurs par compteur (si disponible)
top_sites = df['nom_du_compteur'].value_counts().nlargest(10).index
plt.figure(figsize=(12,6))
sns.boxplot(x='nom_du_compteur', y='comptage_horaire', data=df[df['nom_du_compteur'].isin(top_sites)])
plt.title("Comptage horaire par compteur (Top 10)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 15. Pairplot des variables temporelles et du comptage
sns.pairplot(df[['heure', 'semaine', 'comptage_horaire']])
plt.suptitle("Relation entre heure, semaine et comptage", y=1.02)
plt.show()

In [ ]:
# 19. Histogramme des enregistrements par semaine de l'année
plt.figure(figsize=(12,6))
df['semaine'].value_counts().sort_index().plot(kind='bar', color='skyblue')
plt.title("Nombre d'enregistrements par semaine")
plt.xlabel("Semaine")
plt.ylabel("Nombre d'enregistrements")
plt.show()

In [ ]:
# 20. Évolution du comptage horaire moyen par mois
plt.figure(figsize=(12,6))
mois_order = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
df['mois'] = pd.Categorical(df['mois'], categories=mois_order, ordered=True)
df.groupby('mois')['comptage_horaire'].mean().plot(marker='o')
plt.title("Comptage horaire moyen par mois")
plt.xlabel("Mois")
plt.ylabel("Comptage horaire moyen")
plt.grid(True)
plt.show()

In [ ]:
# Ordre des jours
jours_ordre = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df['jour'] = pd.Categorical(df['jour'], categories=jours_ordre, ordered=True)

# Moyenne du comptage par jour
moyennes = df.groupby('jour')['comptage_horaire'].mean()

# Affichage du graphique
plt.figure(figsize=(10,6))
bars = plt.bar(moyennes.index, moyennes.values, color='coral')
plt.title("Comptage horaire moyen par jour de la semaine")
plt.xlabel("Jour")
plt.ylabel("Comptage horaire moyen")
plt.ylim(0, moyennes.max() * 1.15)

# Affichage des valeurs exactes pour mardi et jeudi
for bar, jour, val in zip(bars, moyennes.index, moyennes.values):
    if jour in ['Tuesday', 'Thursday']:
        plt.text(bar.get_x() + bar.get_width()/2, val + 1, f"{val:.1f}", ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 23. Densité du comptage horaire
plt.figure(figsize=(10,6))
sns.kdeplot(df['comptage_horaire'], shade=True)
plt.title("Densité du comptage horaire")
plt.xlabel("Comptage horaire")
plt.ylabel("Densité")
plt.show()


In [ ]:
# 24. Diagramme en boîte du comptage horaire par heure
plt.figure(figsize=(14,6))
sns.boxplot(x='heure', y='comptage_horaire', data=df)
plt.title("Distribution du comptage horaire par heure")
plt.xlabel("Heure")
plt.ylabel("Comptage horaire")
plt.show()

In [ ]:
# 25. Clustering (KMeans) sur latitude et longitude
if 'latitude' in df.columns and 'longitude' in df.columns:
    scaler = StandardScaler()
    coords_scaled = scaler.fit_transform(df[['latitude','longitude']])
    kmeans = KMeans(n_clusters=5, random_state=42).fit(coords_scaled)
    df['cluster'] = kmeans.labels_
    plt.figure(figsize=(10,6))
    sns.scatterplot(x='longitude', y='latitude', hue='cluster', data=df, palette='tab10')
    plt.title("Clustering géographique des sites de comptage")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.legend(title='Cluster')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Comptage total par heure
heure_total = df.groupby('heure')['comptage_horaire'].sum().sort_values(ascending=False).head(10)

# Affichage du graphique
plt.figure(figsize=(10,6))
bars = plt.bar(heure_total.index.astype(str), heure_total.values, color='orange')

# Titre et axes
plt.title("Top 10 des heures avec le plus fort comptage total")
plt.xlabel("Heure")
plt.ylabel("Comptage total")

# ✅ Retirer notation scientifique
ax = plt.gca()
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x):,}'.replace(',', ' ')))

# ✅ Ajuster l'échelle manuellement
plt.ylim(0, heure_total.max() * 1.15)
plt.yticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()


In [ ]:
# 28. Variation horaire par station (Top 5 pour lisibilité)
top5_stations = df['nom_du_compteur'].value_counts().nlargest(5).index
plt.figure(figsize=(14,6))
sns.lineplot(data=df[df['nom_du_compteur'].isin(top5_stations)], x='heure', y='comptage_horaire', hue='nom_du_compteur', estimator='mean')
plt.title("Évolution horaire du comptage par station (Top 5)")
plt.xlabel("Heure")
plt.ylabel("Comptage horaire moyen")
plt.legend(title='Station')
plt.show()


In [ ]:
# 17. Répartition cumulée du comptage horaire par heure
hourly_sum = df.groupby('heure')['comptage_horaire'].sum().sort_index()
hourly_cumsum = hourly_sum.cumsum()
plt.figure(figsize=(12,6))
plt.plot(hourly_cumsum, marker='o')
plt.title("Comptage horaire cumulé par heure")
plt.xlabel("Heure")
plt.ylabel("Cumul du comptage horaire")
plt.grid(True)
plt.show()

In [ ]:
# 17. Répartition cumulée du comptage horaire par heure
hourly_sum = df.groupby('heure')['comptage_horaire'].sum().sort_index()
hourly_cumsum = hourly_sum.cumsum()
plt.figure(figsize=(12,6))
plt.plot(hourly_cumsum, marker='o')
plt.title("Comptage horaire cumulé par heure")
plt.xlabel("Heure")
plt.ylabel("Cumul du comptage horaire")
plt.grid(True)
plt.show()

In [ ]:
# 18. Heatmap comptage horaire par heure et jour
pivot_table = df.pivot_table(index='heure', columns='jour', values='comptage_horaire', aggfunc='mean')
plt.figure(figsize=(12,8))
sns.heatmap(pivot_table, annot=True, fmt=".1f", cmap="viridis")
plt.title("Moyenne du comptage horaire par heure et jour")
plt.xlabel("Jour de la semaine")
plt.ylabel("Heure")
plt.show()


In [ ]:
# 31. Heatmap moyenne du comptage horaire par heure et station (Top 5)
top5 = df['nom_du_compteur'].value_counts().nlargest(5).index
pivot = df[df['nom_du_compteur'].isin(top5)].pivot_table(index='heure', columns='nom_du_compteur', values='comptage_horaire', aggfunc='mean')
plt.figure(figsize=(10,6))
sns.heatmap(pivot, cmap='coolwarm', annot=True, fmt=".0f")
plt.title("Moyenne horaire du comptage par station (Top 5)")
plt.ylabel("Heure")
plt.xlabel("Station")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

# Données
top_stations = df.groupby('nom_du_compteur')['comptage_horaire'].sum().nlargest(10).sort_values()

# Affichage
plt.figure(figsize=(12, 6))
sns.barplot(x=top_stations.values, y=top_stations.index, palette='magma_r')  # couleurs inversées
plt.title("Top 10 des stations les plus fréquentées (comptage total)")
plt.xlabel("Comptage horaire total")
plt.ylabel("Nom de la station")

# Axe x lisible (espaces entre milliers)
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f"{int(x):,}".replace(',', ' ')))

plt.tight_layout()
plt.show()










In [ ]:
# 31. Heatmap moyenne du comptage horaire par heure et station (Top 5)
if 'nom_du_compteur' in df.columns:
    top5 = df['nom_du_compteur'].value_counts().nlargest(5).index
    pivot = df[df['nom_du_compteur'].isin(top5)].pivot_table(index='heure', columns='nom_du_compteur', values='comptage_horaire', aggfunc='mean')
    plt.figure(figsize=(10,6))
    sns.heatmap(pivot, cmap='coolwarm', annot=True, fmt=".0f")
    plt.title("Moyenne horaire du comptage par station (Top 5)")
    plt.ylabel("Heure")
    plt.xlabel("Station")
    plt.tight_layout()
    plt.show()

In [ ]:
# 1. Histogramme comptage horaire
plt.figure(figsize=(10,6))
sns.histplot(df['comptage_horaire'], bins=30, kde=True)
plt.title("Distribution du comptage horaire")
plt.xlabel("Comptage horaire")
plt.ylabel("Fréquence")
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
df.set_index('Date et heure de comptage').resample('W')['Comptage horaire'].mean().plot()
plt.title("Évolution hebdomadaire du comptage horaire moyen")
plt.xlabel("Date")
plt.ylabel("Comptage horaire moyen")
plt.grid(True)
plt.show()

In [ ]:
import squarify

site_total = df.groupby('Nom du compteur')['Comptage horaire'].sum().sort_values(ascending=False).head(30)
plt.figure(figsize=(14, 8))
squarify.plot(sizes=site_total.values, label=site_total.index, alpha=.8)
plt.title("Treemap des compteurs les plus fréquentés")
plt.axis('off')
plt.show()
#Voir si l'on peut rajouter des aggrégations (pour identifier les zones les plus fréquentées , ça peut etre un arrondissement 

In [ ]:
pivot = df.pivot_table(index='Nom du site de comptage', columns='Heure', values='Comptage horaire', aggfunc='mean').fillna(0)

plt.figure(figsize=(14, 8))
sns.heatmap(pivot, cmap='magma', linewidths=0.5)
plt.title("Trafic moyen par site et heure")
plt.xlabel("Heure")
plt.ylabel("Site")
plt.show()

In [ ]:
hourly_max = df.groupby('Heure')['Comptage horaire'].max()
plt.figure(figsize=(12, 5))
sns.lineplot(x=hourly_max.index, y=hourly_max.values)
plt.title("Pic de fréquentation par heure de la journée")
plt.xlabel("Heure")
plt.ylabel("Comptage horaire max")
plt.grid(True)
plt.show()

In [ ]:
df['Saison'] = df['Date et heure de comptage'].dt.month % 12 // 3 + 1
df['Saison'] = df['Saison'].map({1: 'Hiver', 2: 'Printemps', 3: 'Été', 4: 'Automne'})
profile = df.groupby(['Saison', 'Heure'])['Comptage horaire'].mean().unstack(0)

profile.plot(figsize=(14, 6))
plt.title("Profil horaire moyen selon la saison")
plt.xlabel("Heure")
plt.ylabel("Comptage moyen")
plt.grid(True)
plt.show()
#Voir les décalages entre les heures d'été et d'hiver

In [ ]:
site_perf = df.groupby(['Nom du compteur', 'Latitude', 'Longitude'])['Comptage horaire'].mean().reset_index()
quantiles = site_perf['Comptage horaire'].quantile([0.33, 0.66]).values

def get_color(val):
    if val <= quantiles[0]: return 'green'
    elif val <= quantiles[1]: return 'orange'
    return 'red'

m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=12)
for _, row in site_perf.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=7,
        color=get_color(row['Comptage horaire']),
        fill=True,
        fill_opacity=0.7,
        tooltip=f"{row['Nom du compteur']} : {row['Comptage horaire']:.1f}"
    ).add_to(m)
m.save("carte_perf_sites.html")

In [ ]:
df['Jour_type'] = df['Jour'].apply(lambda x: 'Week-end' if x in ['Saturday', 'Sunday'] else 'Semaine')
sns.boxplot(x='Jour_type', y='Comptage horaire', data=df)
plt.title("Comparaison semaine vs week-end")
plt.show()

In [ ]:
df_ts = df.set_index('Date et heure de comptage').resample('D')['Comptage horaire'].sum().dropna()
df_ts.plot(figsize=(14, 6), title="Évolution journalière du trafic cycliste")

In [ ]:
mean = df_ts.mean()
std = df_ts.std()
anomalies = df_ts[(df_ts < mean - 2 * std) | (df_ts > mean + 2 * std)]

plt.figure(figsize=(14, 6))
df_ts.plot(label="Trafic")
anomalies.plot(style='ro', label="Anomalies")
plt.axhline(mean, color='green', linestyle='--', label='Moyenne')
plt.title("Anomalies du trafic cycliste")
plt.legend()
plt.show()

print("Jours détectés comme anomalies :")
print(anomalies)

In [ ]:
print("\n--- Statistiques descriptives du 'Comptage horaire' (si numérique) ---")
print(df['Comptage horaire'].describe())

print("\n--- Nombre d'enregistrements par site de comptage ---")
print(df['Nom du compteur'].value_counts())

print("\n--- Nombre d'enregistrements par identifiant de compteur ---")
print(df['Identifiant du compteur'].value_counts())

print("\n--- Nombre de sites de comptage uniques ---")
print(f"Nombre de Nom du compteur uniques: {df['Nom du compteur'].nunique()}")
print(f"Nombre de Identifiant du compteur uniques: {df['Identifiant du compteur'].nunique()}")

In [ ]:
# 4.1. Distribution du nombre d'enregistrements ('Comptage horaire' si numérique)
plt.figure(figsize=(10, 6))
sns.histplot(df['Comptage horaire'], bins=50, kde=True)
plt.title('Distribution des valeurs de "Comptage horaire"')
plt.xlabel('Valeur de "Comptage horaire"')
plt.ylabel('Fréquence')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
df['Date d\'installation du site de comptage'].dt.year.value_counts().sort_index().plot(kind='bar')
plt.title('Nombre d\'installations de sites par année')
plt.xlabel('Année d\'installation')
plt.ylabel('Nombre de sites installés')
plt.xticks(rotation=45)
plt.show()